# O que é um Agent criado com LLM?

Um agent no LangChain é um loop controlado onde um LLM recebe um objetivo, pode usar ferramentas, observa os resultados e decide quando parar.


# Loop básico de funcionamento de um 'Agent'
 1. Recebe a pergunta (goal)
 2. Pensa (reasoning / planning)
 3. Decide uma ação (tool ou resposta)
 4. Executa a ação
 5. Observa o resultado
 6. Avalia: já resolvi?
    - se NÃO → volta ao passo 2
    - se SIM → responde e encerra



In [5]:
from dotenv import load_dotenv
load_dotenv()

from langchain.agents import create_agent

# tool é uma função que um agente pode executar. Pode ser qualquer função que quisermos
# Isso significa que podemos dar ao nosso agente infinitas possibilidades de ações que ele pode executar 
# Por exemplo: Consultar um banco de dados, fazer chamadas via api, fazer consultas na web etc
from langchain.tools import tool 

# Interação do agente com mensagens humanas
from langchain_core.messages import HumanMessage 

from langchain_openai import ChatOpenAI


## Tool-augmented LLM 

Um Tool-Augmented LLM é um LLM que pode chamar ferramentas externas, mas não decide sozinho o fluxo completo da tarefa. Ele executa tools quando solicitado, mas não controla um ciclo de decisão completo.

In [6]:
# Biblioteca para buscas na web 
from tavily import TavilyClient
tavily = TavilyClient()

@tool 
def search(query:str) -> str:
    """
    Ferramenta que faz pesquisas na internet 

    Args:
    query: 
        A busca que queremos fazer

    Returns: 
        O reusltado da pesquisa 
    """
    print(f"Buscando por {query}")

    return tavily.search(query=query)


# ReAct Agent (Reason + Act) 

Um ReAct Agent é um sistema que usa um LLM dentro de um loop de decisão (Reason → Act → Observe), podendo chamar tools múltiplas vezes até atingir um objetivo.



In [ ]:
from langchain_tavily import TavilySearch

@tool("get_text_length", description="Retorna o comprimento (número de caracteres) de um texto fornecido.")
def get_text_length(text:str) -> int:
    """ Retorna o length do texto em caracteres (após strip simples) """ # essa description é obrigatória para o agente identificar qual tool ele irá usar
    print(f"get_text_length enter with {text=}")
    text = text.strip("'\n").strip('"')
    return len(text)



llm = ChatOpenAI(temperature=0.5, model="gpt-4.1-mini")
tools_list = [TavilySearch(), get_text_length]

# O agente (que contém o LLM e a lista de tools, aqui TavilySearch) decide se precisa chamar ferramentas:
# Se necessário, ele chama TavilySearch() para buscar na web (o tool devolve resultados).

agent = create_agent(model=llm, tools=tools_list) # Agente tem acesso a tools, logo possui capacidade de chamar tools dinamicamente

# O agente recebe os resultados das ferramentas e constrói um prompt para o LLM que inclui:
# - O que o humano pediu.
# - Os resultados das ferramentas (resumos ou URLs).


print("Inicializando agente")
result = agent.invoke({"messages":HumanMessage(content="Qual o clima de São Paulo - SP?")})
result

Inicializando agente


{'messages': [HumanMessage(content='Qual o clima de São Paulo - SP?', additional_kwargs={}, response_metadata={}, id='0adad24a-fb7d-4b50-b5bd-2922714790d5'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 1222, 'total_tokens': 1252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1152}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'id': 'chatcmpl-D0DTxQLcSqL4GhntGrZ9EfMbYBgHK', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bdd4a-5e8c-72c2-bea4-29711959e642-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'clima de São Paulo SP', 'search_depth': 'fast', 'topic': 'general'}, 'id': 'call_Rv498qnCTOlq2A6pdYuLDzQv', 'type': 'tool_ca

In [8]:
result["messages"]

[HumanMessage(content='Qual o clima de São Paulo - SP?', additional_kwargs={}, response_metadata={}, id='0adad24a-fb7d-4b50-b5bd-2922714790d5'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 1222, 'total_tokens': 1252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1152}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'id': 'chatcmpl-D0DTxQLcSqL4GhntGrZ9EfMbYBgHK', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bdd4a-5e8c-72c2-bea4-29711959e642-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'clima de São Paulo SP', 'search_depth': 'fast', 'topic': 'general'}, 'id': 'call_Rv498qnCTOlq2A6pdYuLDzQv', 'type': 'tool_call'}], invalid

In [18]:
import json
result_dict = json.loads(result["messages"][2].content)
result_dict

{'query': 'clima de São Paulo SP',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.clima-amanha.com/estado/sao-paulo/sao-paulo/',
   'title': 'Clima para Amanhã em São Paulo, SP',
   'content': 'Clima para Amanhã em São Paulo, SP Clima para Amanhã | Estado de São Paulo Clima para amanhã em São Paulo, SP Previsão do tempo para amanhã e para os próximos 5 dias ### Clima em São Paulo O clima em São Paulo é classificado como subtropical úmido.  Devido à sua grande população e atividade industrial, a cidade de São Paulo pode enfrentar problemas de poluição do ar, afetando a qualidade do ar em certas épocas do ano. São Paulo é uma cidade grande e diversificada, e diferentes áreas da cidade [...] e é uma grande área verde no coração da cidade.  São Paulo é uma das maiores metrópoles do mundo em termos de população e área urbana. ##### Temperatura em São Paulo Em São Paulo, o clima é classificado como subtropical úmido, e as temperaturas variam a